# Análise adicional
Aqui foi cruzada a informação de gastos das secretarias com das remunerações e conta com novas análises.

## Importando as Bibliotecas

In [13]:
import pandas as pd
import requests
import numpy as np
import html

In [2]:
# Configura o Pandas para mostrar numeros sem unidade cientifica, e com até 2 casas decimais
# pd.set_option('display.float_format', lambda x: '%.2f' % x)x:,.2f
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)

## Comparação site despesas vs API para verificar se é a mesma informação

### Leitura dos CSVs baixados diretamente do site das despesas
CSVs baixados manualmente pela URL https://transparencia.campinas.sp.gov.br/index.php?action=despesa do mes 12 de 2023 e renomeados

In [5]:
df_pagamentos = pd.read_csv('PMC_Despesas_pagamentos_202312.csv', sep=';', encoding='unicode-escape')
df_pagamentos['Valor Transação'] = (
    df_pagamentos['Valor Transação']
    .str.replace('\.','')
    .str[3:]
    .str.replace(',','.')
    .astype(float)
)

C:\Users\Lucas\AppData\Local\Temp\ipykernel_5736\1224925253.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('\.','')


In [6]:
df_pagamentos.shape

(6343, 12)

### Leitura API

In [7]:
# tipotr=(empenho/liquidacao/pagamento)
reqstr = "https://transparencia.campinas.sp.gov.br/index.php?action=ws&mode=getDespesas&ano=2022&mesinicio=12&mestermino=12&tipotr=pagamento"
r = requests.get(reqstr)
x = r.json()
dfapidesp = pd.DataFrame(x)

### <span style="color:green"> Validado: é a mesma informação</span> Comparação da API com o CSV baixado do site das despesas 

In [8]:
dfapidesp['ValorTransacao']

0       28698.73
1       15995.99
2       11191.35
3        6084.78
4        6084.78
          ...   
6336    87792.03
6337     8642.05
6338    14153.23
6339     2749.38
6340       28665
Name: ValorTransacao, Length: 6341, dtype: object

In [9]:
df_pagamentos['Valor Transação']

0      28698.73
1      15995.99
2      11191.35
3       6084.78
4       6084.78
         ...   
6338   87792.03
6339    8642.05
6340   14153.23
6341    2749.38
6342   28665.00
Name: Valor Transação, Length: 6343, dtype: float64

## Api total vs api no detalhe (objetivo: validar se a api total corresponde a soma da api no detalhe)

In [14]:
# Total
# https://transparencia.campinas.sp.gov.br/ws/getDespesasTotal/ano=2018&mesinicio=05&tipotr=empenho&mestermino=05
# tipotr=(empenho/liquidacao/pagamento)
reqstr = "https://transparencia.campinas.sp.gov.br/ws/getDespesasTotal/ano=2022&mesinicio=01&mestermino=12&tipotr=pagamento"
r = requests.get(reqstr)
x = r.json()
dfapitotal = pd.DataFrame(x)

# Detalhe
# https://transparencia.campinas.sp.gov.br/index.php?action=ws&mode=getDespesas&ano=2018&mesinicio=05&mestermino=05&tipotr=pagamento
# tipotr=(empenho/liquidacao/pagamento)
reqstr = "https://transparencia.campinas.sp.gov.br/index.php?action=ws&mode=getDespesas&ano=2022&mesinicio=01&mestermino=12&tipotr=pagamento"
r = requests.get(reqstr)
x = r.json()
dfapidetalhe = pd.DataFrame(x)

# Converter valores para float
dfapidetalhe['ValorTransacao'] = dfapidetalhe['ValorTransacao'].astype(float)
dfapitotal['valorTotal'] = dfapitotal['valorTotal'].astype(float)

# Converter simbolos HTML (~ç^´) dos nomes das secretarias
dfapitotal['grupo']=dfapitotal['grupo'].apply(html.unescape)
dfapidetalhe['UnidadeGestoraDESC']=dfapidetalhe['UnidadeGestoraDESC'].apply(html.unescape)

# Ordenar por valor
dfapitotal = dfapitotal.sort_values('valorTotal', ascending=False)

# Agrupar API no detalhe por secretaria e ordenar por valor
dfapidetalhegroup = dfapidetalhe.groupby(['UnidadeGestoraDESC']).agg(
    soma_valor_detalhe = ('ValorTransacao', 'sum'),
).sort_values('soma_valor_detalhe', ascending=False).reset_index()



# Padronizar colunas e cruzar
dfapidetalhegroup = dfapidetalhegroup.rename({'UnidadeGestoraDESC':'grupo'}, axis=1)
df_api_merged = pd.merge(dfapitotal, dfapidetalhegroup, 'left', 'grupo')
df_api_merged['percentual_detalhe_sob_totalizado'] = ((df_api_merged['soma_valor_detalhe'] / df_api_merged['valorTotal']) - 1)*100

### Comparação

In [15]:
df_api_merged

,tipoTransacao,grupo,valorTotal,soma_valor_detalhe,percentual_detalhe_sob_totalizado
0,pagamento,071000-SECRETARIA MUNICIPAL DE EDUCACAO,1387277142.97,1394761331.53,0.54
1,pagamento,087000-FUNDO MUNICIPAL DE SAUDE,1365998241.11,1367192093.70,0.09
2,pagamento,171000-ENCARGOS GERAIS DO MUNICIPIO,661762043.15,661762043.15,-0.00
3,pagamento,251000-SECRETARIA MUNICIPAL DE SERVIÇOS PÚBLICOS,541177324.39,542451366.80,0.24
4,pagamento,"583000-Rede Mun. Dr.Mario Gatti de Urgência,Emergência e Hospitalar",353141610.51,353378831.50,0.07
5,pagamento,121000-SECRETARIA MUNICIPAL DE TRANSPORTES,169238305.36,169238305.36,0.00
6,pagamento,097200-FUNDO MUN.DE ASSISTENCIA SOCIAL,132179161.57,132314682.78,0.10
7,pagamento,018000-CAMARA MUNICIPAL,111641793.52,111669432.02,0.02
8,pagamento,051000-SECRETARIA MUNICIPAL DE FINANCAS,102121037.17,102129649.00,0.01
9,pagamento,161000-SECRETARIA MUNICIPAL DE COOP.NOS ASSUNTOS DE SEG.PUBLICA,100251986.60,100278469.35,0.03


<span style='color:#337AFF'>Nota-se uma diferença entre a API DespesasTotal e a API Despesas, sendo que a soma dos valores da API Despesas tende a ser maior do que o valor obtido pela API DespesasTotal, por conta disso foi utilizado para a analise a sona dos valores da API despesas, por contemplar mais do valor gasto pela secretaria</span>

### Comparação Despesas x Remunerações com Base de Salarios

In [16]:
# Detalhe
# https://transparencia.campinas.sp.gov.br/index.php?action=ws&mode=getDespesas&ano=2018&mesinicio=05&mestermino=05&tipotr=pagamento
# tipotr=(empenho/liquidacao/pagamento)
reqstr = "https://transparencia.campinas.sp.gov.br/index.php?action=ws&mode=getDespesas&ano=2022&mesinicio=01&mestermino=12&tipotr=pagamento"
r = requests.get(reqstr)
x = r.json()
dfapidetalhe = pd.DataFrame(x)

# Converter valores para float
dfapidetalhe['ValorTransacao'] = dfapidetalhe['ValorTransacao'].astype(float)

# Converter simbolos HTML (~ç^´) dos nomes das secretarias
dfapidetalhe['UnidadeGestoraDESC'] = dfapidetalhe['UnidadeGestoraDESC'].apply(html.unescape)

# dfapidetalhe['anomes'] = dfapidetalhe['Dia'].str[6:10] + dfapidetalhe['Dia'].str[3:5]

# Agrupar API no detalhe por secretaria e ordenar por valor
dfapidetalhegroup = dfapidetalhe.groupby(['UnidadeGestoraDESC']).agg(
    soma_valor_pagamentos = ('ValorTransacao', 'sum'),
).sort_values('soma_valor_pagamentos', ascending=False).reset_index()

# Padronizar colunas e cruzar
dfapidetalhegroup = dfapidetalhegroup.rename({'UnidadeGestoraDESC':'grupo'}, axis=1)

# Correlacionar Secretaria das despesas com as da base de salários
dfapidetalhegroup['Secretaria'] =   np.where(dfapidetalhegroup['grupo'] == '061000-SECRETARIA MUNICIPAL DE GESTÃO E DESENVOLVIMENTO DE PESSOAS', 'Gestão e Desenvolvimento de Pessoas', 
                                    np.where(dfapidetalhegroup['grupo'] == '291000-SECRETARIA MUN. DE COMUNICAÇÃO', 'Comunicação',
                                    np.where(dfapidetalhegroup['grupo'] == '031000-SECRETARIA MUNICIPAL DE ADMINISTRACAO', 'Administração',
                                    np.where(dfapidetalhegroup['grupo'] == '021000-GABINETE DO PREFEITO', 'Gabinete do Prefeito',
                                    # np.where(dfapidetalhegroup['grupo'] == '', 'Gabinete do Prefeito/Governo',
                                    np.where(dfapidetalhegroup['grupo'] == '281000-SECRETARIA MUNICIPALDE GESTÃO E CONTROLE', 'Gestão e Controle',
                                    np.where(dfapidetalhegroup['grupo'] == '051000-SECRETARIA MUNICIPAL DE FINANCAS', 'Finanças',
                                    np.where(dfapidetalhegroup['grupo'] == '221000-SECRETARIA MUNICIPAL DE ESPORTES E LAZER', 'Esportes e Lazer',
                                    np.where(dfapidetalhegroup['grupo'] == '111000-SECRETARIA MUNICIPAL DE CULTURA E TURISMO', 'Cultura e Turismo',
                                    # np.where(dfapidetalhegroup['grupo'] == '', 'Cultura',
                                    np.where(dfapidetalhegroup['grupo'] == '231000-SECRETARIA MUNICIPAL DO VERDE E DO DESENVOLVIMENTO SUSTENTÁV', 'Verde, Meio Ambiente e Desenvolvimento Sustentável',
                                    np.where(dfapidetalhegroup['grupo'] == '261000-Secretaria Munic. de Desenv. Econômico,Tecnologia e Inovação', 'Desenvolvimento Econômico, Tecnologia e Informação',
                                    # np.where(dfapidetalhegroup['grupo'] == '', 'Desenvolvimento Econômico, Social e Turismo',
                                    np.where(dfapidetalhegroup['grupo'] == '301000-Secretaria Municipal de Planejamento e Urbanismo', 'Planejamento e Urbanismo',
                                    np.where(dfapidetalhegroup['grupo'] == '201000-SECRETARIA MUNICIPAL DE INFRAESTRUTURA', 'Infraestrutura',
                                    np.where(dfapidetalhegroup['grupo'] == '071000-SECRETARIA MUNICIPAL DE EDUCACAO', 'Educação',
                                    np.where(dfapidetalhegroup['grupo'] == '583000-Rede Mun. Dr.Mario Gatti de Urgência,Emergência e Hospitalar', 'Rede Mário Gatti',
                                    np.where(dfapidetalhegroup['grupo'] == '087000-FUNDO MUNICIPAL DE SAUDE', 'Saúde',
                                    # np.where(dfapidetalhegroup['grupo'] == '', 'Hospital Dr Mário Gatti',
                                    np.where(dfapidetalhegroup['grupo'] == '121000-SECRETARIA MUNICIPAL DE TRANSPORTES', 'Transportes',
                                    np.where(dfapidetalhegroup['grupo'] == '251000-SECRETARIA MUNICIPAL DE SERVIÇOS PÚBLICOS', 'Serviços Públicos',
                                    # np.where(dfapidetalhegroup['grupo'] == '', 'Serviços Externos em Geral',
                                    np.where(dfapidetalhegroup['grupo'] == '211000-GABINETE DA OUVIDORIA GERAL DO MUNICIPIO', 'Ouvidoria Geral do Município',
                                    np.where(dfapidetalhegroup['grupo'] == '161000-SECRETARIA MUNICIPAL DE COOP.NOS ASSUNTOS DE SEG.PUBLICA', 'Cooperação nos Assuntos de Segurança Pública',
                                    np.where(dfapidetalhegroup['grupo'] == '041000-SECRETARIA MUNICIPAL DE JUSTIÇA', 'Justiça',
                                    np.where(dfapidetalhegroup['grupo'] == '241000-SECRETARIA MUNICIPAL DE TRABALHO E RENDA', 'Trabalho e Renda',
                                    np.where(dfapidetalhegroup['grupo'] == '151000-SECRETARIA MUNICIPAL DE HABITAÇÃO', 'Habitação',
                                    np.where(dfapidetalhegroup['grupo'] == '091100-SECRETARIA MUNICIPAL DE CIDADANIA,ASSISTÊNCIA E INCLUSÃO SOC', 'Assist. Social, pessoa com deficiência, Dir. Hum.', None
                                    # np.where(dfapidetalhegroup['grupo'] == '', 'Direitos da Pessoa com Deficiência e Cidadania', None
))))))))))))))))))))))) # ))))))

# Base de Salários
df = pd.read_csv('base_unida_tratada_completa.csv')
df = df[df['Ano'] == 2022]
dfsalarios = df.groupby(['Secretaria']).agg(
    salario_bruto_total = ('Bruto recebido', 'sum'),
    salario_bruto_medio = ('Bruto recebido', 'mean'),
    salario_bruto_mediano = ('Bruto recebido', lambda x: x.quantile(0.5)),
    quantidade = ('Matricula', 'nunique'),
).sort_values('salario_bruto_total', ascending=False).reset_index()

# Filtro e Join
dfapidetalhegroup = dfapidetalhegroup[~dfapidetalhegroup['Secretaria'].isna()]
dfjoin = pd.merge(dfapidetalhegroup, dfsalarios, 'left', 'Secretaria')

dfjoin = dfjoin.rename({
    'soma_valor_pagamentos': 'Pagamentos total',
    'salario_bruto_total': 'Salário total',
    'salario_bruto_medio': 'Salário médio',
    'salario_bruto_mediano': 'Salário mediano',
    'quantidade' : 'Quantidade de funcionários',
}, axis=1)

dfjoin['Percentual despesas com Recursos Humanos'] = dfjoin['Salário total'] / dfjoin['Pagamentos total']

#### Tabela com percentual de despesas com recursos Humanos, salário e quantidade

In [17]:
dfjoin.sort_values('Salário médio', ascending=True)[['Secretaria', 'Pagamentos total', 'Salário total', 'Salário médio', 'Salário mediano', 'Percentual despesas com Recursos Humanos', 'Quantidade de funcionários']]

,Secretaria,Pagamentos total,Salário total,Salário médio,Salário mediano,Percentual despesas com Recursos Humanos,Quantidade de funcionários
0,Educação,1394761331.53,451463841.40,6989.15,6034.34,0.32,6078
7,"Assist. Social, pessoa com deficiência, Dir. Hum.",82087866.33,45862150.47,7285.49,6429.09,0.56,584
1,Saúde,1367192093.70,364845998.29,7460.00,5771.55,0.27,4309
6,Cooperação nos Assuntos de Segurança Pública,100278469.35,63169176.50,8279.05,7530.86,0.63,651
2,Serviços Públicos,542451366.80,62861905.88,8429.92,7396.32,0.12,667
13,Esportes e Lazer,32948430.35,15496751.51,8911.30,8037.81,0.47,171
12,Gestão e Desenvolvimento de Pessoas,34483846.05,19112148.87,9215.12,7640.21,0.55,201
19,Trabalho e Renda,9972191.70,5266168.58,10107.81,8814.71,0.53,49
9,Cultura e Turismo,58856708.88,20176621.01,10185.07,9219.66,0.34,259
3,Rede Mário Gatti,353378831.50,217061471.14,10191.64,7544.64,0.61,1849


#### Extra: Todos os grupos das despesas

In [18]:
dfapidetalhegroup

,grupo,soma_valor_pagamentos,Secretaria
0,071000-SECRETARIA MUNICIPAL DE EDUCACAO,1394761331.53,Educação
1,087000-FUNDO MUNICIPAL DE SAUDE,1367192093.70,Saúde
3,251000-SECRETARIA MUNICIPAL DE SERVIÇOS PÚBLICOS,542451366.80,Serviços Públicos
4,"583000-Rede Mun. Dr.Mario Gatti de Urgência,Emergência e Hospitalar",353378831.50,Rede Mário Gatti
5,121000-SECRETARIA MUNICIPAL DE TRANSPORTES,169238305.36,Transportes
8,051000-SECRETARIA MUNICIPAL DE FINANCAS,102129649.00,Finanças
9,161000-SECRETARIA MUNICIPAL DE COOP.NOS ASSUNTOS DE SEG.PUBLICA,100278469.35,Cooperação nos Assuntos de Segurança Pública
10,"091100-SECRETARIA MUNICIPAL DE CIDADANIA,ASSISTÊNCIA E INCLUSÃO SOC",82087866.33,"Assist. Social, pessoa com deficiência, Dir. Hum."
13,021000-GABINETE DO PREFEITO,61086824.63,Gabinete do Prefeito
14,111000-SECRETARIA MUNICIPAL DE CULTURA E TURISMO,58856708.88,Cultura e Turismo
